- For this simulation, we will use the specparam Python package. Further information regarding the model details can be found at: https://specparam-tools.github.io/.

In [ ]:
# Packages 
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from scipy.signal.windows import dpss
from nitime.algorithms.spectral import multi_taper_psd
from mne.time_frequency import psd_array_multitaper
from numpy.lib.stride_tricks import sliding_window_view

from specparam import SpectralModel

from utils.signal_processing import *
from utils.fpp_simulation import *
from utils.plotting import *

---
##### **Characterizing Signal Dynamics Through Their Frequency Content**
This section explores the frequency domain representation of the signal. The Power Spectral Density (PSD) allows us to quantify how the signal's power is distributed across different frequencies, providing a complementary perspective to the temporal dynamics captured by the $\Psi$ operator.

- It is important to remember that an FPP is generated by the convolution of the spike train $s(t)$ (which determines event timing) and the kernel function $k(t)$ (which determines the event shape).

$$
V(t) = s(t) * k(t)
$$

- According to the Convolution Theorem, this implies that the PSD of the FPP, $P_V(f)$, is equivalent to the product of the PSD of the spike train, $P_s(f)$, and the PSD of the kernel function, $P_k(f)$.

$$
P_V(f) = P_s(f) \cdot P_k(f)
$$

In [ ]:
#

In [ ]:
#

As we can observe, the underlying kernel of a signal is closely linked to its temporal properties and, consequently, to its frequency content in the power spectral density.

---
##### **Parameterizing the Power Spectral Density (PSD)**

The spectral density is another way to characterize signal dynamics. In this context, noisier and more stochastic signals (with rapid amplitude fluctuations) will exhibit greater power at high frequencies. The *specparam* model can quantify this by estimating the decay of the aperiodic component in the PSD. In this sense:

- Signals with faster temporal dynamics will present higher high-frequency content.
- The negative linear relationship between power and frequency (on a log-log scale) frequently observed in these types of signals will be less steep.
- The *specparam* model is capable of estimating the slope of this negative linear relationship.
- Higher high-frequency content implies a smaller (flatter) spectral slope.

In [ ]:
#

In [ ]:
#